In [96]:
## test nb for fasttext supervised binary classification ##

In [1]:
# clear the workspace
%reset -f

# print list of files in directory
import os
print(os.listdir())

# the base packages
import collections # for the Counter function
import pandas as pd, numpy as np, re
import sklearn.metrics

# utilities options
randomseed = 1 # the value for the random state used at various points in the pipeline
pd.options.display.max_rows = 1000 # specify if you want the full output in cells rather the truncated list
pd.options.display.max_columns = 200
# to display multiple outputs in a cell without usin print/display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

['.ipynb_checkpoints', '1_12_52_25_10_2018.pickle', '1_t_h2o.csv', '1_v_h2o.csv', '2_12_53_25_10_2018.pickle', '2_t_h2o.csv', '2_v_h2o.csv', '3_12_55_25_10_2018.pickle', '3_t_h2o.csv', '3_v_h2o.csv', 'april_17.csv', 'aug_16.csv', 'aug_17.csv', 'automl_new.ipynb', 'dec_16.csv', 'fasttext example.ipynb', 'feb_17.csv', 'I3_LighGBM_plus_Lime - Reduced Factors-v4.ipynb', 'jan_17.csv', 'july_16.csv', 'july_17.csv', 'JUNE18_TURNOVER.xlsx', 'june_16.csv', 'june_17.csv', 'MAIN_new.ipynb', 'mar_17.csv', 'may_17.csv', 'NEW.rar', 'nov_16.csv', 'nov_17.csv', 'oct_16.csv', 'oct_17.csv', 'plot_document_classification_20newsgroups.ipynb', 'sep_16.csv', 'sep_17.csv', 'test.txt', 'test_dfs.csv', 'TRAIN.csv', 'train.txt', 'Turnover Report NAZ 2017 Year End.csv', 'turnover-2016-final.csv', 'VALID.csv']


In [234]:
## find and append multiple dataframes of the type specified in string
def append_datasets(cols_to_remove, string = ['TRAIN', 'VALID']):
    # pass either train or valid as str argument
    temp_files = [name for name in os.listdir() if name.startswith(string)]
    temp_dict = {}
    for i in temp_files:
        df_name = re.sub(string=i, pattern='.csv', repl='')
        print(df_name)
        temp_dict[df_name] = pd.read_csv(i, na_values=['No Data', ' ', 'UNKNOWN', ''])
        temp_dict[df_name].columns = map(str.lower, temp_dict[df_name].columns)
        temp_dict[df_name].drop(cols_to_remove, axis = 1, inplace = True)
        chars_to_remove = [' ', '.', '(', ')', '__', '-']
        for i in chars_to_remove:
            temp_dict[df_name].columns = temp_dict[df_name].columns.str.strip().str.lower().str.replace(i, '_')
    temp_list = [v for k,v in temp_dict.items()]
    temp = pd.concat(temp_list, axis=0, sort=True, ignore_index=True)
    return temp

In [235]:
# input/ouput files

cols_to_remove = ['global id']
train = append_datasets(string='TRAIN', cols_to_remove=cols_to_remove)
test = append_datasets(string='VALID', cols_to_remove=cols_to_remove)

text_cols = ['short_text_of_organizational_unit', 'position_text', 'abinbev_entity4', 'physical_work_location_description',
            'physical_work_location_description', 'physical_work_location_city', 'physical_work_location_city', 
            'manager_position_desc', 'costcenter_description', 'macro_entity4', 'abinbev_entity3',
            'local_entity_description', 'pers_subarea_text', 'group', 'abinbev_entity2', 'global_job', 'job_family',
            'functional_area']
train_text = train[text_cols]
test_text = test[text_cols]
train_text['string_all'] = train_text[text_cols].apply(lambda x: ' '.join(x.dropna()), axis=1)
test_text['string_all'] = test_text[text_cols].apply(lambda x: ' '.join(x.dropna()), axis=1)
train_text = train_text[['string_all']]
test_text = test_text[['string_all']]
y_train = train[['label']]
y_test = test[['label']]

train_final = pd.concat([y_train.reset_index(drop=True), train_text.reset_index(drop=True)], axis=1)
test_final = pd.concat([y_test.reset_index(drop=True), test_text.reset_index(drop=True)], axis=1)

TRAIN
VALID


C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [236]:
train_final['string'] = '__label__' + train_final['label'].astype('str') + ' ' + train_final['string_all']
test_final['string'] = '__label__' + test_final['label'].astype('str') + ' ' + test_final['string_all']

train_final.drop(['label', 'string_all'], inplace=True, axis=1)
test_final.drop(['label', 'string_all'], inplace=True, axis=1)

In [237]:
train_final.to_csv('train.txt', encoding='utf-8', index=False, header=False, sep='\t')
test_final.to_csv('test.txt', encoding='utf-8', index=False, header=False, sep='\t')

In [238]:
from fastText import train_supervised

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [239]:
model = train_supervised(input = 'train.txt', epoch=10000, lr=0.01, wordNgrams=6, verbose=10, minCount=1, loss='softmax')

In [242]:
print_results(*model.test('test.txt', k=1))

N	5734
P@1	0.928
R@1	0.928


In [243]:
test_len = test_final.shape[0]

predict_list = [None] * test_len
iter = 0
for i in range(test_len):
    x=model.predict(test_text.iloc[iter][0], k=2)
    y=pd.DataFrame(list(x))
    y.columns=y.iloc[0]
    y=y.reindex(y.index.drop(0))
    predict_list[iter] = float(y['__label__1'])
    iter+=1

In [245]:
sklearn.metrics.roc_auc_score(y_score=predict_list, y_true=y_test)

0.6748679919571519